# Zillow:
<hr style="border:2px solid black">

## Goal/Ask:
We want to be able to predict the property tax assessed values ('taxvaluedollarcnt') of **Single Family Properties** that had a transaction during 2017.

We have a model already, but we are hoping your insights can help us improve it. **I need recommendations on a way to make a better model. Maybe you will create a new feature out of existing ones that works better, try a non-linear regression algorithm, or try to create a different model for each county. Whatever you find that works (or doesn't work) will be useful.** Given you have just joined our team, we are excited to see your outside perspective.

One last thing, Maggie lost the email that told us where these properties were located. Ugh, Maggie :-/. Because property taxes are assessed at the county level, we would like to know what states and counties these are located in.
<hr style="border:2px solid black">


# Imports


In [449]:
# Standard ds imports:
import pandas as pd
import numpy as np

# Viz
import matplotlib.pyplot as plt
import seaborn as sns

# Data Prep
from sklearn.model_selection import train_test_split
import sklearn.preprocessing

# Statistics
from scipy import stats

# Modeling

# Model evaluation

# Custom Moduls
import os
import env
import acquire as a

<hr style="border:2px solid black">

# **Acquire**
Plan --> **Acquire** --> Prepare --> Explore --> Model --> Deliver

- Single Family Properties 2017
- look at data
	- describe, info, head, shape
- understand what your data means
	- know what each column is
	- know what your target variable is
- [ ] Add to data dictionary
- [ ] acquire.py modularization


In [450]:
# Acquire
zillow = a.get_zillow_data()

In [451]:
# Make copy of df for manipulation
z = zillow.copy()

In [452]:
# drop duplicate rows
z = z.drop_duplicates()
# drop duplicate columns
z = z.T.drop_duplicates().T

In [453]:
# make var to save dropped column list
columns_to_drop = ['id',
 'parcelid',
 'airconditioningtypeid',
 'architecturalstyletypeid',
 'buildingclasstypeid',
 'buildingqualitytypeid',
  'decktypeid',
  'heatingorsystemtypeid',
 'propertylandusetypeid',
  'storytypeid',
  'typeconstructiontypeid',
  'structuretaxvaluedollarcnt',
  'taxamount',
  'calculatedbathnbr',
  'finishedfloor1squarefeet',
   'finishedsquarefeet12',
 'finishedsquarefeet50',
 'finishedsquarefeet6',
 'fullbathcnt',
  'pooltypeid10',
 'pooltypeid2',
 'pooltypeid7',
  'pooltypeid10',
 'pooltypeid2',
 'pooltypeid7',
  'propertycountylandusecode',
  'rawcensustractandblock',
  'propertyzoningdesc',
  'roomcnt',
  'threequarterbathnbr',
  'fireplaceflag',
  'taxdelinquencyyear',
   'censustractandblock',
   'parcelid.1',
  'landtaxvaluedollarcnt',
  'storydesc',
  'propertylandusedesc',
  'typeconstructiondesc',
  'basementsqft',
  'fireplacecnt',
  'garagecarcnt',
  'hashottuborspa',
  'poolsizesum',
  'regionidneighborhood',
  'regionidcity',
  'unitcnt',
  'yardbuildingsqft17',
  'yardbuildingsqft26',
  'architecturalstyledesc',
  'transactiondate',
  'garagetotalsqft',
  'airconditioningdesc',
  'heatingorsystemdesc'
  ]

In [454]:
z.columns.to_list()

['id',
 'parcelid',
 'airconditioningtypeid',
 'architecturalstyletypeid',
 'basementsqft',
 'bathroomcnt',
 'bedroomcnt',
 'buildingclasstypeid',
 'buildingqualitytypeid',
 'calculatedbathnbr',
 'decktypeid',
 'finishedfloor1squarefeet',
 'calculatedfinishedsquarefeet',
 'finishedsquarefeet12',
 'finishedsquarefeet50',
 'finishedsquarefeet6',
 'fips',
 'fireplacecnt',
 'fullbathcnt',
 'garagecarcnt',
 'garagetotalsqft',
 'hashottuborspa',
 'heatingorsystemtypeid',
 'latitude',
 'longitude',
 'lotsizesquarefeet',
 'poolcnt',
 'poolsizesum',
 'pooltypeid10',
 'pooltypeid2',
 'pooltypeid7',
 'propertycountylandusecode',
 'propertylandusetypeid',
 'propertyzoningdesc',
 'rawcensustractandblock',
 'regionidcity',
 'regionidcounty',
 'regionidneighborhood',
 'regionidzip',
 'roomcnt',
 'storytypeid',
 'threequarterbathnbr',
 'typeconstructiontypeid',
 'unitcnt',
 'yardbuildingsqft17',
 'yardbuildingsqft26',
 'yearbuilt',
 'numberofstories',
 'fireplaceflag',
 'structuretaxvaluedollarcnt',
 

In [455]:
z = z.drop(columns=columns_to_drop)

In [456]:
rename_columns ={
'bathroomcnt':'baths',
'bedroomcnt':'beds',
'calculatedfinishedsquarefeet':'sqft',
'fips':'county',
'hashottuborspa':'spatub',
'lotsizesquarefeet':'lotsqft',
'numberofstories':'stories',
'taxvaluedollarcnt':'tax_value',
'taxdelinquencyflag':'tax_delinquency',
'airconditioningdesc':'ac_type',
'heatingorsystemdesc':'heat_type'}

z = z.rename(columns=rename_columns)

In [457]:
# save column names to list variable
z_columns = z.columns.to_list()


In [458]:
z.info()

<class 'pandas.core.frame.DataFrame'>
Index: 56079 entries, 0 to 56078
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   baths            56079 non-null  object
 1   beds             56079 non-null  object
 2   sqft             55848 non-null  object
 3   county           56079 non-null  object
 4   latitude         56079 non-null  object
 5   longitude        56079 non-null  object
 6   lotsqft          55729 non-null  object
 7   poolcnt          10951 non-null  object
 8   regionidcounty   56079 non-null  object
 9   regionidzip      55979 non-null  object
 10  yearbuilt        55830 non-null  object
 11  stories          15493 non-null  object
 12  tax_value        56071 non-null  object
 13  assessmentyear   56079 non-null  object
 14  tax_delinquency  1129 non-null   object
 15  logerror         56079 non-null  object
dtypes: object(16)
memory usage: 9.3+ MB


In [459]:
z.describe().T

,count,unique,top,freq
baths,56079.0,24.0,2.0,24535.0
beds,56079.0,13.0,3.0,25018.0
sqft,55848.0,4684.0,1200.0,136.0
county,56079.0,3.0,6037.0,37287.0
latitude,56079.0,53910.0,33708500.0,12.0
longitude,56079.0,54384.0,-117756000.0,12.0
lotsqft,55729.0,14365.0,6000.0,1132.0
poolcnt,10951.0,1.0,1.0,10951.0
regionidcounty,56079.0,3.0,3101.0,37287.0
regionidzip,55979.0,382.0,97118.0,509.0


In [460]:
z.head()

,baths,beds,sqft,county,latitude,longitude,lotsqft,poolcnt,regionidcounty,regionidzip,yearbuilt,stories,tax_value,assessmentyear,tax_delinquency,logerror
0,0.0,0.0,NaN,6037.0,34140430.0,-118625364.0,4083.0,NaN,3101.0,96337.0,NaN,NaN,27516.0,2015.0,NaN,0.055619
1,0.0,0.0,NaN,6037.0,34585014.0,-118162010.0,11975.0,NaN,3101.0,97329.0,NaN,NaN,10.0,2016.0,NaN,0.008669
2,0.0,0.0,NaN,6037.0,34563376.0,-118019104.0,9403.0,NaN,3101.0,97330.0,NaN,NaN,10.0,2016.0,NaN,-0.021896
3,0.0,0.0,NaN,6037.0,34526913.0,-118050581.0,3817.0,NaN,3101.0,97330.0,NaN,NaN,2108.0,2016.0,NaN,-0.017167
4,2.0,4.0,3633.0,6037.0,34560018.0,-118169806.0,9826.0,NaN,3101.0,97329.0,2005.0,NaN,296425.0,2016.0,NaN,0.042463


<hr style="border:2px solid black">

# **Prepare**
Plan --> Acquire --> **Prepare** --> Explore --> Model --> Deliver

**Prepare Actions:**
- clean the data
	- handle nulls
	- handle outliers
	- correct datatypes

- Preprocessing:
	- encode variables
	- split into train, validate/, test
	- scale data (after train/validate/test split) -- Preprocessing
- document how your changing the data
#### wrangle_exercises
- prepare.py


In [461]:
#confirm whole numbers
#for col in z.columns:
#    print(col)
#    print(z[col].value_counts().sort_index())
#    print()

In [462]:
col = z.columns.to_list()

#are we dealing with any nulls?
num_missing = z[col].isna().sum()
num_missing

baths                  0
beds                   0
sqft                 231
county                 0
latitude               0
longitude              0
lotsqft              350
poolcnt            45128
regionidcounty         0
regionidzip          100
yearbuilt            249
stories            40586
tax_value              8
assessmentyear         0
tax_delinquency    54950
logerror               0
dtype: int64

# ** Finish handling nulls. Impute the values you actually want for for poolcnt **

In [463]:
##### handle nulls #####

z = z.fillna(0)

# impute for poolcnt, stories, tax_delinquency

# Specify columns where you want to replace 0 with "no"
columns_to_replace = ['poolcnt', 'stories', 'tax_delinquency']
# Replace 0 with "no" in the specified columns
z[columns_to_replace] = z[columns_to_replace].applymap(lambda x: 'None' if x == 0 else x)

# drop the rest of the nulls
z = z[~(z == 0).any(axis=1)]

In [464]:
col = z.columns.to_list()

#are we dealing with any nulls?
num_missing = z[col].isna().sum()
num_missing

baths              0
beds               0
sqft               0
county             0
latitude           0
longitude          0
lotsqft            0
poolcnt            0
regionidcounty     0
regionidzip        0
yearbuilt          0
stories            0
tax_value          0
assessmentyear     0
tax_delinquency    0
logerror           0
dtype: int64

In [465]:
z.shape

(55318, 16)

In [419]:
df_replaced.poolcnt.value_counts()

poolcnt
0.0    45128
1.0    10951
Name: count, dtype: int64

In [467]:
# correct dtypes
make_ints = ['beds','tax_value', 'yearbuilt', 'sqft', 'county', 'latitude', 'longitude', 'lotsqft', 'poolcnt', 'regionidzip', 'stories', 'assessmentyear']
make_float = ['baths','logerror']

for col in make_ints:
    z[col] = z[col].astype(int)
for col in make_float:
    z[col] = z[col].astype(float)

ValueError: invalid literal for int() with base 10: 'None'

In [261]:
z.dtypes

baths              float64
beds                 int64
sqft                 int64
county               int64
garagetotalsqft      int64
latitude             int64
longitude            int64
lotsqft              int64
poolcnt              int64
regionidcounty     float64
regionidzip          int64
yearbuilt            int64
stories              int64
tax_value            int64
assessmentyear       int64
tax_delinquency     object
logerror           float64
heat_type           object
ac_type             object
dtype: object

In [ ]:
# handle outliers

<hr style="border:2px solid black">

# **Explore**
Plan --> Acquire --> Prepare --> **Explore** --> Model --> Deliver
### Initial Hypotheses:
1. one
2. two
3. three
### Vizualize:
1. **Univariate analysis (looking at only one variable)**
	- [ ] Plot out the distributions of each feature. _This is critical b/c many of our statisitical tools and machine learning algorithms assume certain distributions. If your data isn't remotely normally distributed, then avoid using any tools that assume normally distributed data.

>Takeaways:

2. **Bi-Multivariate Distributions**
- [ ] Plot out the interaction of 2 or more variables.
- [ ] Plot out how subgroups compare to each-other and to the overall population.
- [ ] Document any surprises you may find in visualizing. This means write down your takeaways; documenting your takeaways is a huge component of your final deliverable/analysis.

### Hypothesis Testing:
- [ ] <u> <b>Identify features that correlate with the target variable.</b></u>
- [ ] <u> <b>Identify features that correlate with each other. If feature A and feature B are each tightly correlated with the target variable, but they're also tightly correlated with each other, we should use one feature that correlates better, rather than use both. </b> </u>

#### Feature Engineering
- Create any custom features
	* add to data dictionary

-  Select Potential Features: (based on results of hypothesis testing)
	1. Relevent Feature 1:
	2. 2
	3. 3
	
- From those features select the best using RFE, SelectKBest

## Exploration Summary
All of the features investigated were found to be statistically significant and will be used in our model.
<u><b>Drivers of Churn:</b></u>
* Monthly Charges (Especially above $70)
* Contract Type (M-M)
* Payment Type (Manual)
* Internet Service Type (Fiber Optic)
* Tech Support (For Fiber Optic customers)
<br>

<hr style="border:2px solid black">

# **Modeling**
Plan --> Acquire --> Prepare --> Explore --> **Model** --> Deliver

- [ ] Select Model types:
	- [ ] OLS
	- [ ] LassoLars
	- [ ] GLM
	- [ ] Polynomial


## <u>Evaluate:</u>
- [ ] Baseline
- [ ] Different features for each model
- [ ] Best performing of each category to other categories


# <u>Iterate:</u>
#### Am I done with clean-up/prep? Taken care of outliers?
#### Tune Model
#### Feature Engineering:


